## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [3]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

C:\Users\reyna\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [7]:

classifier=Sequential()

#卷積組合
input_shape = (32, 32, 3)
classifier.add(Convolution2D(filters=32,kernel_size=(3,3),padding='same', input_shape=input_shape, activation='relu'))#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(BatchNormalization())

#'''自己決定MaxPooling2D放在哪裡'''
classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Convolution2D(filters=32,kernel_size=(3,3),padding='same', activation='relu'))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size=(2,2)))

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(128, activation='relu')) #output_dim=100,activation=relu

#輸出
classifier.add(Dense(output_dim=10,activation='softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=10)

C:\Users\reyna\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`


Epoch 1/10
50000/50000 [==============================] - 173s 3ms/step - loss: 1.3071 - accuracy: 0.5420
Epoch 2/10
50000/50000 [==============================] - 175s 4ms/step - loss: 0.9059 - accuracy: 0.6819
Epoch 3/10
50000/50000 [==============================] - 174s 3ms/step - loss: 0.7456 - accuracy: 0.7390
Epoch 4/10
50000/50000 [==============================] - 190s 4ms/step - loss: 0.6174 - accuracy: 0.7837
Epoch 5/10
50000/50000 [==============================] - 228s 5ms/step - loss: 0.5056 - accuracy: 0.8225
Epoch 6/10
50000/50000 [==============================] - 229s 5ms/step - loss: 0.3982 - accuracy: 0.8602
Epoch 7/10
50000/50000 [==============================] - 218s 4ms/step - loss: 0.3095 - accuracy: 0.8921
Epoch 8/10
50000/50000 [==============================] - 234s 5ms/step - loss: 0.2250 - accuracy: 0.9235
Epoch 9/10
50000/50000 [==============================] - 197s 4ms/step - loss: 0.1815 - accuracy: 0.9370
Epoch 10/10
50000/50000 [=====================

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [8]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[2.6699884e-03, 2.0073116e-05, 2.3169285e-03, 2.5183980e-03,
        8.8435000e-01, 2.6665479e-08, 1.1579097e-02, 2.0373736e-08,
        9.6545458e-02, 6.0870378e-11]], dtype=float32)